# Red Wine Draft 7

# Libraries

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings
import datetime
import sqlite3
import imblearn


from IPython.display import display
from pylab import rcParams
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier

# Load Raw Data

In [11]:
redwine = pd.read_csv(
    "C:\\Users\\Space\\Documents\\py\\Projects\\TuringCollege\\RedWine\\DataSets\\winequality-red.csv",
    index_col=False,
)

In [12]:
redwine['quality'] = redwine['quality'].replace([8], 1)
redwine['quality'] = redwine['quality'].replace([3, 4, 5, 6, 7], 0)

In [13]:
# Load libraries
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation


feature_cols = [
    "citric acid",
    "residual sugar",
    "free sulfur dioxide",
    "total sulfur dioxide",
    "density",
    "sulphates",
    "alcohol",
]

X = redwine[feature_cols]
y = redwine.quality

In [14]:
oversample = RandomOverSampler(sampling_strategy='minority')
oversample = RandomOverSampler(sampling_strategy='auto')
steps = [('over', RandomOverSampler()), ('model', DecisionTreeClassifier())]
pipeline = Pipeline(steps=steps)
X, y = oversample.fit_resample(X, y)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) # 70% training and 30% test

In [16]:
model = LogisticRegression(solver='liblinear', random_state=0)
clf = model.fit(X, y)
model.score(X, y)

0.8962681846932321

In [17]:
confusion_matrix(y, model.predict(X))

array([[1253,  328],
       [   0, 1581]], dtype=int64)